In [1]:
import time
import numpy as np
import pandas as pd
import torch

import torch.optim as optim

In [2]:
from src.utils_data import M4TS
from src.utils_data import get_m4_all_series

from src.utils_config import ModelConfig
from src.utils_visualization import plot_prediction

from src.utils_train import SmylLoss
# from src.utils_models import VanillaLSTM
# from src.utils_models import ES
from src.utils_models import ES, RNN, ESRNN

In [3]:
mc = ModelConfig(config_file='configs/config_m4quarterly.yaml',
                 root_dir='./results/')

all_series = get_m4_all_series(mc=mc, data='train')
# train(mc, all_series)

# model = ESRNN(mc)
# print("\n Model parameters: \n", model.rnn.parameters, "\n")

In [4]:
def train(mc, all_series):
    print(10*'='+' Training {} '.format(mc.dataset_name) + 10*'=')
    #torch.set_default_tensor_type(torch.float32)
    torch.autograd.set_detect_anomaly(True)
    
    # Random Seeds
    torch.manual_seed(mc.copy)
    np.random.seed(mc.copy)

    # Model
    esrnn = ESRNN(mc)

    # Optimizers
    es_optimizer = optim.Adam(params=esrnn.es.parameters(),
                              lr=mc.learning_rate*mc.per_series_lr_multip, 
                              betas=(0.9, 0.999), eps=mc.gradient_eps)

    rnn_optimizer = optim.Adam(params=esrnn.rnn.parameters(),
                         lr=mc.learning_rate,
                         betas=(0.9, 0.999), eps=mc.gradient_eps)
    
    # Loss Functions
    smyl_loss = SmylLoss(tau=mc.tau,
                         level_variability_penalty=mc.level_variability_penalty)

    # training code
    for epoch in range(mc.max_epochs):
        start = time.time()
        forecast_losses = []
        lev_variability_losses = []
        state_losses = []
        
        for j in range(len(all_series)):
            es_optimizer.zero_grad()
            rnn_optimizer.zero_grad()

            ts_object = all_series[j]
            windows_y, windows_y_hat, levels = esrnn(ts_object)
            
            loss = smyl_loss(windows_y, windows_y_hat, levels)
            loss.backward()
            torch.nn.utils.clip_grad(esrnn.rnn.parameters,
                                     max_norm=mc.gradient_clipping_threshold)
            torch.nn.utils.clip_grad(esrnn.es.parameters,
                                     max_norm=mc.gradient_clipping_threshold)
            rnn_optimizer.step()
            es_optimizer.step()
        
        if epoch >= (mc.max_epochs-mc.averaging_level):
            copy = epoch+mc.averaging_level-mc.max_epochs
            esrnn.save(copy=copy)

        print("========= Epoch {} finished =========".format(epoch))
        print("Training time: {}".format(time.time()-start))
        print("Forecast loss: {}".format(np.mean(forecloss_ex.npvalue())))

    print('Train finished!')

In [5]:
train(mc, all_series)

========== Training m4quarterly ==========
input_data.dtype torch.float32


/Users/distiller/project/conda/conda-bld/pytorch_1565272679438/work/torch/csrc/autograd/python_anomaly_mode.cpp:57: UserWarning: Traceback of forward call that caused the error:
  File "/Users/kin_gutierrez/miniconda/envs/esrnn_torch/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/Users/kin_gutierrez/miniconda/envs/esrnn_torch/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/Users/kin_gutierrez/miniconda/envs/esrnn_torch/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/Users/kin_gutierrez/miniconda/envs/esrnn_torch/lib/python3.6/site-packages/traitlets/config/application.py", line 664, in launch_instance
    app.start()
  File "/Users/kin_gutierrez/miniconda/envs/esrnn_torch/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 563, in start
    self.io_loop.start()
  File "/Users/kin_gutierrez/miniconda/envs/esrnn_torch/lib/python3.6/site-p

RuntimeError: one of the variables needed for gradient computation has been modified by an inplace operation: [torch.FloatTensor [1, 29]], which is output 0 of SliceBackward, is at version 25; expected version 24 instead. Hint: the backtrace further above shows the operation that failed to compute its gradient. The variable in question was changed in there or anywhere later. Good luck!

In [ ]:
model = RNN(mc)
print("\n Model parameters: \n", model.rnn.parameters, "\n")

In [ ]:
np.float32(ver)